# Stack-Augmented Recurrent Neural Networks

In [1]:
# Import libraries and relevant dependencies
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import string
from torch.autograd import Variable

# Dyck library
from tasks.dyck_generator import DyckLanguage

# RNN Models
from models.rnn_models import VanillaRNN, SRNN_Softmax, SRNN_Softmax_Temperature, SRNN_GumbelSoftmax

# Set default tensor type "double"
torch.set_default_tensor_type('torch.DoubleTensor')

#### Fix the random seed:

In [2]:
randomseed_num = 23
print ('RANDOM SEED: {}'.format(randomseed_num))
random.seed (randomseed_num)
np.random.seed (randomseed_num)
torch.manual_seed(randomseed_num)

RANDOM SEED: 23


#### GPU/CPU Checkpoint

In [ ]:
# GPU/CPU Check
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## GPU stuff
print (device)

## Dyck Languages (Don't run this section if not using)

A probabilistic context-free grammar for $\mathcal{D}_n$ can be written as follows:

\begin{align*}
S \rightarrow \begin{cases} 
(_i\, S\, )_i & \text{with probability } \frac{p}{n} \\
S\,S & \text{with probability } q \\ 
\varepsilon & \text{with probability } 1 - (p+q) 
\end{cases}
\end{align*}
where $0 < p, q < 1$ and $p+q < 1$.

### Training and Test Corpora Generation

Training corpus window : [`MIN_SIZE`, `MAX_SIZE`]

Test corpus window: [`MAX_SIZE+2`, `2*MAX_SIZE`]

In [ ]:
## Parameters of the Probabilistic Dyck Language 
NUM_PAR = 2
MIN_SIZE = 2
MAX_SIZE = 50
P_VAL = 0.5
Q_VAL = 0.25

# Number of samples in the training corpus
TRAINING_SIZE = 5000
# Number of samples in the test corpus
TEST_SIZE = 5000

# Create a Dyck language generator
Dyck = DyckLanguage (NUM_PAR, P_VAL, Q_VAL)
all_letters = word_set = Dyck.return_vocab ()
n_letters = vocab_size = len (word_set)

print('Loading data...')

training_input, training_output, st = Dyck.training_set_generator (TRAINING_SIZE, MIN_SIZE, MAX_SIZE)
test_input, test_output, st2 = Dyck.training_set_generator (TEST_SIZE, MAX_SIZE + 2, 2 * MAX_SIZE)

for i in range (1):
    print (training_output[i])
    print (Dyck.lineToTensor(training_output[i]))
    print (Dyck.lineToTensorSigmoid(training_output[i]))

## Infix to Postfix Data Generation

In [ ]:
# Number of samples in the training corpus
TRAINING_SIZE = 5000
# Number of samples in the test corpus
TEST_SIZE = 5000

In [3]:
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def update(self, item):
        self.items[len(self.items) - 1] = item
        return self.items

    def peek(self):
        return self.items[len(self.items) - 1]

    def size(self):
        return len(self.items)

In [4]:
import pandas as pd

df = pd.read_csv("dataset.tsv.txt", sep="\t",header=None)[0:TRAINING_SIZE+TEST_SIZE]
df[0] = df[0].str.replace(" ","") 
df[1] = df[1].str.replace(" ","")
df = df.drop(columns=[2])

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.tsv.txt'

In [ ]:
df

In [6]:
def infixToPostfix(infixexpr):
    axns_ohe = []
    axns_str = []
    
    prec = {}
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec["-"] = 2
    prec["("] = 1
    opStack = Stack()
    postfixList = []
    tokenList = [char for char in infixexpr]

    for token in tokenList:
#         print(token, end=" ")
        token_vec = [0, 0, 0]  # push, pop, no-op
        token_str = ""
        if token not in prec and token != ")":
            postfixList.append(token)
            token_vec[2] += 1
            token_str += "2"
        elif token == '(':
            opStack.push(token)
            token_vec[0] += 1
            token_str += "0"
        elif token == ')':
            topToken = opStack.pop()
            token_vec[1] += 1
            token_str += "1"
            while topToken != '(':
                postfixList.append(topToken)
                topToken = opStack.pop()
#                 token_vec[1] += 1
        else:
            while (not opStack.isEmpty()) and (prec[opStack.peek()] >= prec[token]):
                postfixList.append(opStack.pop())
                token_vec[1] += 1
            opStack.push(token)
            token_vec[0] += 1
            token_str += "0"
        axns_ohe.append(token_vec)
        axns_str.append(token_str)
    
    while not opStack.isEmpty():
        postfixList.append(opStack.pop())
        
    return "".join(axns_str), axns_ohe, "".join(postfixList)

In [10]:
def ohe_axn(df):
    axn_list_list = []
    for item in df.index:
        axn_list_list.append(infixToPostfix(df.iloc[item, 0])[0])
        
    return axn_list_list

In [ ]:
eqn_vocab = ['(', ')', '*', '+', '-', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
axn_vocab = ["0", "1", "2"]

def lineToTensor(line, n_letters, vocab):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][vocab.index(letter)] = 1.0
    return tensor

In [8]:
lineToTensor("0002021021021", len(axn_vocab), axn_vocab)

NameError: name 'lineToTensor' is not defined

In [ ]:
lineToTensor("(((9*9)/9)-9)", len(eqn_vocab), eqn_vocab)

In [9]:
df["axn_list"] = ohe_axn(df)

NameError: name 'df' is not defined

In [ ]:
df

In [ ]:
# define the input and output in the same way as the dyck languages
training_input, training_output = df[:TRAINING_SIZE][0].tolist(), df[:TRAINING_SIZE]["axn_list"].tolist()
test_input, test_output = df[TRAINING_SIZE:TRAINING_SIZE+TEST_SIZE][0].tolist(), df[TRAINING_SIZE:TRAINING_SIZE+TEST_SIZE]["axn_list"].tolist()

In [ ]:
print(len(training_input), len(test_input))
print(len(training_output), len(test_output))

### Stack-RNN  Parameters

In [ ]:
# Number of hidden units
n_hidden = 8
# Number of hidden layers
n_layers = 1
# Stack size
stack_size = 104
stack_dim = 1

In [ ]:
## Stack-RNN with Softmax
# model = SRNN_Softmax (n_hidden, vocab_size, vocab_size, n_layers, stack_size, stack_dim).to(device)
# model = VanillaRNN(n_hidden, vocab_size, vocab_size).to(device)  # works with dyck
model = VanillaRNN(n_hidden, len(axn_vocab), len(eqn_vocab)).to(device)  # works with predicting stack actions
# model = SRNN_Softmax (n_hidden, vocab_size, vocab_size, n_layers, stack_size, stack_dim).to(device)

# Learning rate
learning_rate = .01
# Minimum Squared Error (MSE) loss
criterion = nn.MSELoss() 
# Adam optimizer (https://arxiv.org/abs/1412.6980)
optim = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
print ('Model details:')
print (model)

In [ ]:
# Number of epochs to train our model
epochs = 2
# Output threshold
epsilon = 0.5

### Training and Testing the Stack-RNN Model

In [ ]:
def test_model (model, data_input, data_output, which):
    # Turn on the eval mode
    model.eval()
    # Total number of "correctly" predicted samples
    correct_num = 0
    with torch.no_grad():
        for i in range (len(data_output)):
            len_input = len (data_input[i])
            model.zero_grad ()
            # Initialize the hidden state
            hidden = model.init_hidden()
            # Initialize the stack
            stack = torch.zeros (stack_size, stack_dim).to(device)
            # Target values
            if which == "train":
                target = lineToTensor(training_output[i], len(axn_vocab), axn_vocab).to(device) 
            else:
                target = lineToTensor(test_output[i], len(axn_vocab), axn_vocab).to(device) 
            # Output values
            output_vals = torch.zeros (target.shape)
            
            for j in range (len_input):
                if which == "train":
                    output, hidden, stack = model (lineToTensor(training_input[i][j], len(eqn_vocab), eqn_vocab).to(device), hidden, stack)
                else:
                    output, hidden, stack = model (lineToTensor(test_input[i][j], len(eqn_vocab), eqn_vocab).to(device), hidden, stack)
                output_vals [j] = output

            # Binarize the entries based on the output threshold
            out_np = np.int_(output_vals.detach().numpy() >= epsilon)
            target_np = np.int_(target.detach().numpy())
            
            # (Double-)check whether the output values and the target values are the same
            if np.all(np.equal(out_np, target_np)) and (out_np.flatten() == target_np.flatten()).all():
                # If so, increase `correct_num` by one
                correct_num += 1
                
    return float(correct_num)/len(data_output) * 100, correct_num

In [ ]:
def train (model, optimizer, criterion, epoch_num=2):
    # Turn on the train model for the model
    model.train()
    # Arrays for loss and "moving" accuracy per epoch
    loss_arr = []
    correct_arr = []
    for epoch in range(1, epoch_num + 1):
        print ('Epoch: {}'.format(epoch))
        
        # Total loss per epoch
        total_loss = 0
        # Total number of "correctly" predicted samples so far in the epoch
        counter = 0

        for i in range (TRAINING_SIZE):
            len_input = len (training_input[i])
            # Good-old zero grad
            model.zero_grad ()
            # Initialize the hidden state
            hidden = model.init_hidden()
            # Initialize the stack 
            stack = torch.zeros (stack_size, stack_dim).to(device)
            # Target values
            target = lineToTensor(training_output[i], len(axn_vocab), axn_vocab).to(device) 
            # Output values
            output_vals = torch.zeros (target.shape)

            for j in range (len_input):
                output, hidden, stack = model (lineToTensor(training_input[i][j], len(eqn_vocab), eqn_vocab).to(device), hidden, stack)
                output_vals [j] = output
            
            # MSE (y, y_bar)
            loss = criterion (output_vals, target)
            # Add the current loss to the total loss
            total_loss += loss.item()
            # Backprop! 
            loss.backward ()
            optimizer.step ()
            
            # Print the performance of the model every 500 steps
            if i % 250 == 0:
                print ('Sample Number {}: '.format(i))
                print ('Input : {}'.format(training_input[i]))
                print ('Output: {}'.format(training_output[i]))
                print ('* Counter: {}'.format(counter))
                print ('* Avg Loss: {}'.format(total_loss/(i+1))) 

            # Binarize the entries based on the output threshold
            out_np = np.int_(output_vals.detach().numpy() >= epsilon)
            target_np = np.int_(target.detach().numpy())
                
            # "Moving" training accuracy
            if np.all(np.equal(out_np, target_np)) and (out_np.flatten() == target_np.flatten()).all():
                counter += 1
                
            # At the end of the epoch, append our total loss and "moving" accuracy
            if i == TRAINING_SIZE - 1:
                print ('Counter: {}'.format(float(counter)/TRAINING_SIZE))
                loss_arr.append (total_loss)
                correct_arr.append(counter) 

        if epoch % 1 == 0:
            print ('Training Accuracy %: ', correct_arr)
            print ('Loss: ', loss_arr)

#### Let there be light!

In [ ]:
train (model, optim, criterion, epoch_num=epochs)

### Evaluate the Performance of the Stack-RNN Model

In [ ]:
# Training set accuracy 
correct_num = test_model (model, training_input, training_output, "train")
print ('Training accuracy: {}.'.format(correct_num))

In [ ]:
# Test set accuracy 
correct_num = test_model (model, test_input, test_output, "test")
print ('Test accuracy: {}.'.format(correct_num))

### Save/Upload the Model Weights

In [ ]:
# Save the model weights
torch.save(model.state_dict(), 'models/vanilla_rnn_model_weights.pth')

In [ ]:
# # Load the model weights
# model.load_state_dict(torch.load('models/stack_rnn_model_weights.pth'))

## Visualize the Stack Elements

In [ ]:
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
import plotly.graph_objs as go
import math

import seaborn as sns; sns.set()

import matplotlib
import matplotlib.pyplot as plt

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)

### Get the Hidden State and Stack Configuration for a Given Input

In [ ]:
def get_hidden_and_stack_info (model, input, output):
    # Turn on the evaluation mode for the model
    model.eval()
    # Hidden state values
    hidden_states = []
    # Stack configuration
    stack_config = []
    # Stack operation weights
    operation_weights = []
    # Most recently pushed element to the stack
    new_elt_inserted = []
    
    with torch.no_grad():
        len_input = len (input)
        model.zero_grad ()
        
        # Initialize the hidden state
        hidden = model.init_hidden()
        # Initalize the stack configuration
        stack = torch.zeros (stack_size, stack_dim).to(device)
        # Target values
        target = Dyck.lineToTensorSigmoid(output)
        # Output values
        output_vals = torch.zeros (target.shape).to(device)

        for j in range (len_input):
            # Feed the input to the model
            output, hidden, stack = model (Dyck.lineToTensor(input[j]).to(device), hidden, stack)
            # Hidden state values
            hidden_states.append (hidden.cpu().numpy())
            # Stack configuration 
            stack_config.append (stack.cpu().numpy())
            # Stack operation weights
            operation_weights.append (model.action_weights.cpu().numpy())
            # New element inserted to the stack
            new_elt_inserted.append (model.new_elt.cpu().numpy())
            # Output value
            output_vals [j] = output.view(-1)
        
        # Binarize the entries based on the output threshold
        out_np = np.int_(output_vals.cpu().detach().numpy() >= epsilon)
        target_np = np.int_(target.cpu().detach().numpy())
        
        # (Double-)check whether the output values and the target values are the same
        if np.all(np.equal(out_np, target_np)) and (out_np.flatten() == target_np.flatten()).all():
            print ('Correct!')
        else:
            print ('Incorrect')
            
    return hidden_states, stack_config, operation_weights, new_elt_inserted

In [ ]:
input_seq = '([])[[[((([[(())]])))]]][()]'
output_seq =  Dyck.output_generator (input_seq)

In [ ]:
hidden_states, stack_config, operation_weights, new_elt_inserted = get_hidden_and_stack_info (model, input_seq, output_seq)

### Visualize the Stack Operation Weights at Each Timestep

In [ ]:
def visualize_stack_operation_weights (operation_weights, input_seq, timestep=0):
    # Stack operation labels
    labels = ['PUSH', 'POP']
    stack_op_weights = np.squeeze(operation_weights)
    plt.figure(figsize=(16, 5))
    fig = sns.heatmap(stack_op_weights.T, cmap=sns.light_palette("#34495e"),xticklabels=input_seq, yticklabels=labels, vmin=0, vmax=1)
    fig.set_title('Strength of Stack Operations at Each Timestep', fontsize=17)
    cbar = fig.collections[0].colorbar
    cbar.set_ticks(np.linspace(0,1,6))
    plt.xlabel('Sequence', fontsize=16)
    plt.ylabel('Actions', fontsize=16)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=14)
    plt.show()
    # plt.savefig('stackrnn_weights', dpi=128, bbox_inches='tight')

In [ ]:
visualize_stack_operation_weights (operation_weights, input_seq)

### Visualize the Stack Configuration at Each Timestep

In [ ]:
def visualize_stack_configuration (stack_config, input, dimension=0):
    stack_bound = 13 #len (input)
    print (np.array(stack_config).shape)
    print (stack_bound)
    stack_config = np.round(np.array(stack_config)[:, :stack_bound+1, dimension], decimals=3)
    location = np.arange (1, stack_bound+2)
    plt.figure(figsize=(18, 12))
    fig = sns.heatmap(stack_config.T, cmap='viridis', yticklabels = location, xticklabels=input, annot=True, cbar=False)
    fig.invert_yaxis()
    fig.set_title('Stack Entries at Each Timestep', fontsize=17)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.xlabel('Sequence', fontsize=16)
    plt.ylabel('Stack Location', fontsize=16)
    plt.show()

In [ ]:
visualize_stack_configuration (stack_config, input_seq, 0)

### Visualize the Hidden State Values at Each Timestep

In [ ]:
def visualize_hidden_states (hidden_states, input):
    plt.style.use('default')
    domain = list(range(len(input)))
    hidden_states = np.squeeze(hidden_states).T
    for i in range (int(n_hidden/2)):
        plt.figure()
        for j in range (2):
            plt.plot (domain, hidden_states[i*2+j], label='Unit {}'.format(i*2+j+1))
        plt.legend (loc='upper right')
        plt.xticks(domain, input_seq) 
        plt.title ('Analysis of the Hidden State Dynamics')
        plt.ylabel ('Activation Values')
        plt.ylim (-1.15, 1.15, 10)
        plt.show()

In [ ]:
visualize_hidden_states (hidden_states, input_seq)

In [ ]:
# Q.E.D.